<a href="https://colab.research.google.com/github/8Dis-like/UCLALearning/blob/main/CM146_Discussion_4_Pytorch_intro_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Adapted from https://towardsdatascience.com/logistic-regression-on-mnist-with-pytorch-b048327f8d19
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

## About
 A simple binary image classification example: classify bewteen bees and not bees. Figure is from https://colab.research.google.com/github/dair-ai/notebooks/blob/master/_notebooks/2020-03-18-pytorch_logistic_regression.ipynb.
![alt text](https://drive.google.com/uc?export=view&id=11Bv3uhZtVgRVYVWDl9_ZAYQ0GU36LhM9)


In this tutorial, we are going to implement a logistic regression model from scratch with PyTorch. The model will be designed with neural networks in mind and will be used for a simple image classification task.



Now let's see can we work on implement another image classification problem: a very popular  multi-class classification problem on MNIST benchmark dataset.

## Step 1. Load Dataset

In [ ]:
# To load the dataset, we make use of torchvision.datasets,
# a library which has almost all the popular datasets used in Machine Learning.
# You can check out the complete list of datasets at https://pytorch.org/docs/stable/torchvision/datasets.html.

# For first time set: download=True
# set the path "root" where to save the preprossed data after download'
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 36.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.10MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.35MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.62MB/s]


In [ ]:
# Set hyperparams
batch_size = 100
n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784
output_dim = 10
lr_rate = 0.001


## Step 2. Make Dataset Iterable

In [ ]:
# Make Dataset Iterable
# A common practice is to shuffle trainset but not the testset
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Step 3. Create Model Class

In [ ]:
# Create the Model Class
# Must have the forward func and super class must be torch.nn.Module
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x) # x.shape (batch_size x input_dim)
        return outputs # outputs.shape (batch_size x output_dim)

## Step 4. Instantiate Model Class

In [ ]:
# Instantiate the Model Class
model = LogisticRegression(input_dim, output_dim)

## Step 5. Instantiate Loss Class

In [ ]:
# Instantiate the Loss Class
# We use the cross-entropy to compute the loss.
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then NLL loss

## Step 6. Instantiate Optimizer Class

In [ ]:
# Instatnitate the Optimizer Class
# The optimizer will be the learning algorithm we use. In this case, we will use the Stochastic Gradient Descent.
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

## Step 7. Train Model

In [ ]:
# Last Step: Train the Model
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):

        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images) # batch_size x output_dim
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%500==0:
            # calculate Test Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                # print(labels)
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Train Loss: {}. test Accuracy: {}.".format(iter, loss.item(), accuracy))

Iteration: 500. Train Loss: 1.9060405492782593. test Accuracy: 70.23999786376953.
Iteration: 1000. Train Loss: 1.5646954774856567. test Accuracy: 77.95999908447266.
Iteration: 1500. Train Loss: 1.3653838634490967. test Accuracy: 80.5.
Iteration: 2000. Train Loss: 1.125389814376831. test Accuracy: 81.80999755859375.
Iteration: 2500. Train Loss: 1.118086814880371. test Accuracy: 82.55999755859375.
Iteration: 3000. Train Loss: 0.9977609515190125. test Accuracy: 83.36000061035156.
